In [1]:
import pandas as pd

In [2]:
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [3]:
df = pd.read_csv('Insights.csv')
df.columns = ['Insight', 'InsightID', 'InsightParentID', 'IsParent']
df.head()

,Insight,InsightID,InsightParentID,IsParent
0,"Les intérimaires manquent de rigueur en termes de ponctualité, de professionnalisme, et d'attention au détail, ce qui se traduit par des erreurs coûteuses et un comportement inapproprié au travail.",1702251558470x613868062724425100,1702251592643x358642939349547600,non
1,"Les candidats indiquent ne pas être informés concernant l'entreprise, son activité et les détails des postes avant l'entretien, créant un manque d'adéquation et de préparation.",1702251576702x588656886216548500,1702252292076x775565084647336700,non
2,Le client suggère la mise en place de contrôles de références pour les nouveaux intérimaires n'ayant jamais travaillé avec l'agence.,1702251591177x115416254019377940,NaN,non
3,"Les intérimaires présentent des lacunes en termes de discipline au travail, impactant la qualité de leur performance et leur conformité aux processus.",1702251592643x358642939349547600,NaN,oui
4,Il est nécessaire d'améliorer l'exactitude des déclarations SI/SIR en fonction des expositions et risques associés à chaque poste de travail pour une meilleure traçabilité et un suivi efficace de la santé des intérimaires tout au long et après leur carrière.,1702251599635x333691042597010760,NaN,non


In [4]:
df[filter_no_parents]['InsightParentID']

NameError: name 'filter_no_parents' is not defined

In [ ]:
#!pip install --upgrade pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 11.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.1
    Uninstalling pandas-1.5.1:
      Successfully uninstalled pandas-1.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
eurlex 0.1.4 requires beautifulsoup4==4.9.3, but you have beautifulsoup4 4.12.2 which is incompatible.


In [5]:
filter_no_parents = df['InsightParentID'] != df['InsightParentID']
df[filter_no_parents]['InsightParentID'] = df[filter_no_parents]['InsightID']
df[filter_no_parents]


,Insight,InsightID,InsightParentID,IsParent
2,Le client suggère la mise en place de contrôles de références pour les nouveaux intérimaires n'ayant jamais travaillé avec l'agence.,1702251591177x115416254019377940,NaN,non
3,"Les intérimaires présentent des lacunes en termes de discipline au travail, impactant la qualité de leur performance et leur conformité aux processus.",1702251592643x358642939349547600,NaN,oui
4,Il est nécessaire d'améliorer l'exactitude des déclarations SI/SIR en fonction des expositions et risques associés à chaque poste de travail pour une meilleure traçabilité et un suivi efficace de la santé des intérimaires tout au long et après leur carrière.,1702251599635x333691042597010760,NaN,non
8,Il y a une suggestion pour que Randstad adopte une politique de paiement d'avance pour travailler avec des entreprises en difficulté financière afin d'éviter de perdre des clients.,1702251633585x517965186747366900,NaN,non
11,Les clients expriment un mécontentement général lié à la mauvaise gestion des rendez-vous et un manque de professionnalisme et de disponibilité du personnel de Randstad.,1702251653260x739912670275253900,NaN,oui
...,...,...,...,...
255,Les clients apprécient la réactivité de l'entreprise en réponse à leurs demandes.,1702253426969x958780167048609200,NaN,non
258,"Les clients valorisent le professionnalisme et souhaitent des services RH modulaires et planifiés, y compris un système de prise de rendez-vous.",1702253449690x342285172891097860,NaN,oui
268,Les clients souhaitent que les intérimaires possèdent une expertise préalable dans leur domaine d'activité spécifique.,1702253561513x883729916898281400,NaN,oui
277,"Les clients estiment que les services de Randstad, y compris les frais de mutuelle, sont perçus comme trop chers comparativement à la concurrence.",1702253667582x240092666822965250,NaN,oui


In [60]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('OrdalieTech/Solon-embeddings-large-0.1')
#model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
#model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
sentiment_model = SentenceTransformer('lxyuan/distilbert-base-multilingual-cased-sentiments-student')


ConnectionError: (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: a186ee53-2203-4cb3-a004-7064c5290647)')

In [30]:
sentences = df['Title']
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(297, 1024)

In [31]:
sentence_embeddings

array([[-0.04266007, -0.01242622,  0.03966972, ..., -0.02307059,
        -0.09413584, -0.03576062],
       [-0.01890397, -0.02291213, -0.00415571, ..., -0.00181556,
        -0.01434652, -0.0462789 ],
       [-0.06531541, -0.00406456, -0.00932761, ..., -0.04511409,
        -0.03815044, -0.04094359],
       ...,
       [-0.03131469,  0.02609637,  0.02687677, ..., -0.02091159,
        -0.06056459, -0.03400551],
       [ 0.00440707, -0.04418156,  0.00804911, ..., -0.02692328,
        -0.07442538, -0.01602571],
       [-0.05931276, -0.0298768 ,  0.00861773, ..., -0.06241287,
        -0.02789943, -0.03819002]], dtype=float32)

In [69]:
#@title Plot the archive {display-mode: "form"}

# UMAP reduces the dimensions from 1024 to 2 dimensions that we can plot
reducer = umap.UMAP(n_neighbors=15)
umap_embeds = reducer.fit_transform(sentence_embeddings)
# Prepare the data to plot and interactive visualization
# using Altair
df_explore = pd.DataFrame(data={'text': df['Title']})
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]


# Plot
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['text']
).properties(
    width=700,
    height=400
)
chart.interactive()

ValueError: Expected a 1D array, got an array with shape (297, 2)

In [47]:
df_explore

,text,x,y
0,"Les intérimaires manquent de rigueur en termes de ponctualité, de professionnalisme, et d'attention au détail, ce qui se traduit par des erreurs coûteuses et un comportement inapproprié au travail.",-0.388733,5.184397
1,"Les candidats indiquent ne pas être informés concernant l'entreprise, son activité et les détails des postes avant l'entretien, créant un manque d'adéquation et de préparation.",0.193930,4.341852
2,Le client suggère la mise en place de contrôles de références pour les nouveaux intérimaires n'ayant jamais travaillé avec l'agence.,-0.444737,2.613282
3,"Les intérimaires présentent des lacunes en termes de discipline au travail, impactant la qualité de leur performance et leur conformité aux processus.",-0.279644,5.280058
4,Il est nécessaire d'améliorer l'exactitude des déclarations SI/SIR en fonction des expositions et risques associés à chaque poste de travail pour une meilleure traçabilité et un suivi efficace de la santé des intérimaires tout au long et après leur carrière.,-0.520634,4.842052
...,...,...,...
292,Le client demande une augmentation de la réactivité de la part de l'entreprise.,2.345634,2.972209
293,Il est suggéré de mettre en place un service disponible 24 heures sur 24.,3.190371,2.583641
294,Les clients sont très satisfaits de la qualité des intérimaires proposés par Randstad.,3.221508,-1.303958
295,"Le client a exprimé une satisfaction générale en déclarant que son expérience s'est ""très bien passée"".",5.624993,0.539423


In [48]:
df_explore["Insight"] = 1

In [55]:
other_sentences = ["Les gens sont mécontents", "Les gens sont très contents", "Le produit est parfait", "Le produit doit être amélioré", "délais", "réactivité", "incohérence"]
other_sentence_embeddings = model.encode(other_sentences)

In [56]:
# Prepare the data to plot and interactive visualization
# using Altair


In [57]:
other_umap_embeds = reducer.transform(other_sentence_embeddings)

other_df = pd.DataFrame({"text":other_sentences, 'x': other_umap_embeds[:,0], 'y': other_umap_embeds[:,1], "Insight": 0})
other_df

,text,x,y,Insight
0,Les gens sont mécontents,2.789261,4.749218,0
1,Les gens sont très contents,5.859206,1.025309,0
2,Le produit est parfait,5.582722,1.125249,0
3,Le produit doit être amélioré,2.335799,2.095050,0
4,délais,-0.298254,5.843427,0
5,réactivité,4.951604,1.774886,0
6,incohérence,1.116359,5.597881,0


In [58]:
df_explore_new = pd.concat([df_explore, other_df])

In [59]:
# Plot
chart = alt.Chart(df_explore_new).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['text'],
    color='Insight',
).properties(
    width=700,
    height=400
)
chart.interactive()

alt.Chart(...)

In [61]:
df_explore_new.head()

,text,x,y,Insight
0,"Les intérimaires manquent de rigueur en termes de ponctualité, de professionnalisme, et d'attention au détail, ce qui se traduit par des erreurs coûteuses et un comportement inapproprié au travail.",-0.388733,5.184397,1
1,"Les candidats indiquent ne pas être informés concernant l'entreprise, son activité et les détails des postes avant l'entretien, créant un manque d'adéquation et de préparation.",0.193930,4.341852,1
2,Le client suggère la mise en place de contrôles de références pour les nouveaux intérimaires n'ayant jamais travaillé avec l'agence.,-0.444737,2.613282,1
3,"Les intérimaires présentent des lacunes en termes de discipline au travail, impactant la qualité de leur performance et leur conformité aux processus.",-0.279644,5.280058,1
4,Il est nécessaire d'améliorer l'exactitude des déclarations SI/SIR en fonction des expositions et risques associés à chaque poste de travail pour une meilleure traçabilité et un suivi efficace de la santé des intérimaires tout au long et après leur carrière.,-0.520634,4.842052,1


In [66]:
df_explore_new['text'].to_list()

["Les intérimaires manquent de rigueur en termes de ponctualité, de professionnalisme, et d'attention au détail, ce qui se traduit par des erreurs coûteuses et un comportement inapproprié au travail.",
 "Les candidats indiquent ne pas être informés concernant l'entreprise, son activité et les détails des postes avant l'entretien, créant un manque d'adéquation et de préparation.",
 "Le client suggère la mise en place de contrôles de références pour les nouveaux intérimaires n'ayant jamais travaillé avec l'agence.",
 'Les intérimaires présentent des lacunes en termes de discipline au travail, impactant la qualité de leur performance et leur conformité aux processus.',
 "Il est nécessaire d'améliorer l'exactitude des déclarations SI/SIR en fonction des expositions et risques associés à chaque poste de travail pour une meilleure traçabilité et un suivi efficace de la santé des intérimaires tout au long et après leur carrière.",
 "Le client apprécie le professionnalisme durant sa visite à V

In [67]:
df_explore_new['embedding'] = reducer.transform(df_explore_new['text'])

ValueError: could not convert string to float: "Les intérimaires manquent de rigueur en termes de ponctualité, de professionnalisme, et d'attention au détail, ce qui se traduit par des erreurs coûteuses et un comportement inapproprié au travail."

In [62]:
umap_embeds

array([[ 2.7892613 ,  4.7492185 ],
       [ 5.859206  ,  1.0253093 ],
       [ 5.5827217 ,  1.1252486 ],
       [ 2.3357992 ,  2.0950499 ],
       [-0.29825407,  5.843427  ],
       [ 4.9516044 ,  1.7748865 ],
       [ 1.1163592 ,  5.597881  ]], dtype=float32)